# Pain in the Net - Laplacian Pyramid Translation Network (LPTN)
Application of Laplacian Pyramid Translation Network (LPTN) to domain adaptation of diffusion MRI.


Code by:

Tyler Spears - tas6hh@virginia.edu

Dr. Tom Fletcher

---

Based on the following work(s):

* `J. Liang, H. Zeng, and L. Zhang, “High-Resolution Photorealistic Image Translation in Real-Time: A Laplacian Pyramid Translation Network,” 2021, pp. 9392–9400. Accessed: Aug. 26, 2021. [Online]. Available: https://openaccess.thecvf.com/content/CVPR2021/html/Liang_High-Resolution_Photorealistic_Image_Translation_in_Real-Time_A_Laplacian_Pyramid_Translation_CVPR_2021_paper.html
`


## Imports & Environment Setup

### Imports

In [ ]:
# Automatically re-import project-specific modules.
%load_ext autoreload
%autoreload 1

# imports
import collections
import functools
import io
import datetime
import time
import math
import itertools
import os
import shutil
import pathlib
import copy
import pdb
import inspect
import random
import subprocess
import sys
import warnings
from pathlib import Path
import typing
import zipfile

import ants
import dipy
import dipy.core
import dipy.reconst
import dipy.reconst.dti
import dipy.segment.mask
import dipy.viz
import dipy.viz.regtools
import dotenv

# visualization libraries
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import IPython

# Try importing GPUtil for printing GPU specs.
# May not be installed if using CPU only.
try:
    import GPUtil
except ImportError:
    warnings.warn("WARNING: Package GPUtil not found, cannot print GPU specs")
from tabulate import tabulate
from IPython.display import display, Markdown
import ipyplot

# Data management libraries.
import nibabel as nib
import natsort
from natsort import natsorted
import addict
from addict import Addict
import pprint
from pprint import pprint as ppr
from tdigest import TDigest

# Computation & ML libraries.
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torchio
import pytorch_lightning as pl
import kornia
import monai

import skimage
import skimage.feature
import skimage.filters
import skimage.measure
import scipy

plt.rcParams.update({"figure.autolayout": True})
plt.rcParams.update({"figure.facecolor": [1.0, 1.0, 1.0, 1.0]})

# Set print options for ndarrays/tensors.
np.set_printoptions(suppress=True, threshold=100, linewidth=88)
torch.set_printoptions(sci_mode=False, threshold=100, linewidth=88)

In [ ]:
# Update notebook's environment variables with direnv.
# This requires the python-dotenv package, and direnv be installed on the system
# This will not work on Windows.
# NOTE: This is kind of hacky, and not necessarily safe. Be careful...
# Libraries needed on the python side:
# - os
# - subprocess
# - io
# - dotenv

# Form command to be run in direnv's context. This command will print out
# all environment variables defined in the subprocess/sub-shell.
command = "direnv exec {} /usr/bin/env".format(os.getcwd())
# Run command in a new subprocess.
proc = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True, cwd=os.getcwd())
# Store and format the subprocess' output.
proc_out = proc.communicate()[0].strip().decode("utf-8")
# Use python-dotenv to load the environment variables by using the output of
# 'direnv exec ...' as a 'dummy' .env file.
dotenv.load_dotenv(stream=io.StringIO(proc_out), override=True);

In [ ]:
# Project-specific scripts
# It's easier to import it this way rather than make an entirely new package, due to
# conflicts with local packages and anaconda installations.
# You made me do this, poor python package management!!
if "PROJECT_ROOT" in os.environ:
    lib_location = str(Path(os.environ["PROJECT_ROOT"]).resolve())
else:
    lib_location = str(Path("../../").resolve())
if lib_location not in sys.path:
    sys.path.insert(0, lib_location)
import lib as pitn

# Include the top-level lib module along with its submodules.
%aimport lib
# Grab all submodules of lib, not including modules outside of the package.
includes = list(
    filter(
        lambda m: m.startswith("lib."),
        map(lambda x: x[1].__name__, inspect.getmembers(pitn, inspect.ismodule)),
    )
)
# Run aimport magic with constructed includes.
ipy = IPython.get_ipython()
ipy.run_line_magic("aimport", ", ".join(includes))

In [ ]:
# torch setup
# allow for CUDA usage, if available
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
# keep device as the cpu
# device = torch.device('cpu')
print(device)

### Specs Recording

In [ ]:
%%capture --no-stderr cap
# Capture output and save to log. Needs to be at the *very first* line of the cell.
# Watermark
%load_ext watermark
%watermark --author "Tyler Spears" --updated --iso8601  --python --machine --iversions --githash
if torch.cuda.is_available():

    # GPU information
    # Taken from
    # <https://www.thepythoncode.com/article/get-hardware-system-information-python>.
    # If GPUtil is not installed, skip this step.
    try:
        gpus = GPUtil.getGPUs()
        print("=" * 50, "GPU Specs", "=" * 50)
        list_gpus = []
        for gpu in gpus:
            # get the GPU id
            gpu_id = gpu.id
            # name of GPU
            gpu_name = gpu.name
            driver_version = gpu.driver
            cuda_version = torch.version.cuda
            # get total memory
            gpu_total_memory = f"{gpu.memoryTotal}MB"
            gpu_uuid = gpu.uuid
            list_gpus.append(
                (
                    gpu_id,
                    gpu_name,
                    driver_version,
                    cuda_version,
                    gpu_total_memory,
                    gpu_uuid,
                )
            )

        print(
            tabulate(
                list_gpus,
                headers=(
                    "id",
                    "Name",
                    "Driver Version",
                    "CUDA Version",
                    "Total Memory",
                    "uuid",
                ),
            )
        )
    except NameError:
        print("CUDA Version: ", torch.version.cuda)

else:
    print("CUDA not in use, falling back to CPU")

In [ ]:
# cap is defined in an ipython magic command
print(cap)

Author: Tyler Spears

Last updated: 2021-09-20T15:11:42.902698+00:00

Python implementation: CPython
Python version       : 3.8.8
IPython version      : 7.23.1

Compiler    : GCC 7.3.0
OS          : Linux
Release     : 5.4.0-84-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit

Git hash: 4281c55c802880d2c05bbb8802c05e0b93820076

ipywidgets       : 7.6.3
nibabel          : 3.2.1
pandas           : 1.2.3
monai            : 0.5.dev2114
dipy             : 1.4.1
skimage          : 0.18.1
json             : 2.0.9
seaborn          : 0.11.1
natsort          : 7.1.1
GPUtil           : 1.4.0
matplotlib       : 3.4.1
numpy            : 1.20.2
sys              : 3.8.8 (default, Feb 24 2021, 21:46:12) 
[GCC 7.3.0]
IPython          : 7.23.1
addict           : 2.4.0
pytorch_lightning: 1.4.5
kornia           : 0.5.8
torch            : 1.9.0
ants             : 0.2.7
torchio          : 0.18.37
scipy            : 1.5.3

=================================================

### Data Variables & Definitions Setup

In [ ]:
# Set up directories
data_dir = pathlib.Path(os.environ["DATA_DIR"]) / "hcp"
assert data_dir.exists()
write_data_dir = pathlib.Path(os.environ["WRITE_DATA_DIR"]) / "hcp"
assert write_data_dir.exists()
results_dir = pathlib.Path(os.environ["RESULTS_DIR"])
assert results_dir.exists()
tmp_results_dir = pathlib.Path(os.environ["TMP_RESULTS_DIR"])
assert tmp_results_dir.exists()

---

## LPTN Generator Testing & Development

In [ ]:
hardi_fname, hardi_bval_fname, hardi_bvec_fname = dipy.data.get_fnames('stanford_hardi')
label_fname = dipy.data.get_fnames('stanford_labels')

data, affine, hardi_img = dipy.io.image.load_nifti(hardi_fname, return_img=True)
# Move channels to first dimension.
data = data.transpose(3, 0, 1, 2)
labels = dipy.io.image.load_nifti_data(label_fname)
bvals, bvecs = dipy.io.gradients.read_bvals_bvecs(hardi_bval_fname, hardi_bvec_fname)
gtab = dipy.core.gradients.gradient_table(bvals, bvecs)

In [ ]:
x = data[0]
x = torch.from_numpy(x).float()
print(x.shape)
divisible_target = 2**3
data_shape = torch.as_tensor(x.shape).float()
print(divisible_target - (data_shape % divisible_target))
pad_pre = torch.floor((divisible_target - (data_shape % divisible_target))/2).int()
pad_pre = reversed(pad_pre.tolist())
pad_post = torch.ceil((divisible_target - (data_shape % divisible_target))/2).int()
pad_post = reversed(pad_post.tolist())
x = F.pad(x, tuple(itertools.chain.from_iterable(zip(pad_pre, pad_post))))
print(torch.as_tensor(x.shape) % divisible_target)

### Laplacian Pyramid Decomposotion + Reconstruction Test

In [ ]:
pyramid = pitn.nn.layers.LaplacePyramid3d(3)
hf, residual = pyramid.decompose(x)
x_recon = pyramid.reconstruct(hf, residual)
print(torch.sqrt(F.mse_loss(x, x_recon[0, 0])))

In [ ]:
plt.imshow(x[:, 50].cpu().numpy())
plt.colorbar()
plt.show()

plt.imshow(x_recon[0, 0, :, 50].cpu().numpy())
plt.colorbar()
plt.show()

plt.imshow(x[:, 50].cpu().numpy() - x_recon[0, 0, :, 50].cpu().numpy())
plt.colorbar()

## LPTN Translation Network Testing

In [ ]:
net = pitn.nn.gan.generative.LPTN(6, 3)
y = net(x.repeat(4, 6, 1, 1, 1))
print(x.shape)
print(y.shape)

In [ ]:
for i in range(3):
    dipy.viz.regtools.overlay_slices(x.detach().numpy(), y[0, 0].detach().numpy(), slice_type=i).set_dpi(120)

## LPTN Discriminator Network Testing

In [ ]:
discrim = pitn.nn.gan.adversarial.MultiDiscriminator(6)
y = discrim(x.repeat(4, 6, 1, 1, 1))
y

In [ ]:
y.shape

In [ ]:
%autoreload